<a href="https://colab.research.google.com/github/akoo-45/ML-hw5-hiring-decision/blob/master/hw5_code_hk2827_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os 
os.chdir('/content')
data_path = "gdrive/My Drive/ML/HW5"
os.chdir(data_path)

In [3]:
# Encoding categorical variables
pip install category_encoders

     |████████████████████████████████| 92kB 5.5MB/s 


In [89]:
# Download Dependencies 
import pandas as pd
import numpy as np
from sklearn import datasets # SKLearn, golden stardard for ML with python
from sklearn import linear_model

# Reading three CSV files 
train_df = pd.read_csv('hw5-trainingset-hk2827.csv')
train_df = pd.read_csv('hw5-testset-hk2827.csv')
cook_df = pd.read_csv('CodeBook-SELECT.csv')

Y = train_df.loc[:, 'job_performance']


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (50,53,54,55,56,256,257,258,268,376) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (50,255,256,257,258,260,268) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
# Preprocessing traian_set

# (1)
# drop overtly useless features
train_df.drop(['cntryid_e','job_performance','row', 'uni'], axis=1, inplace= True) 
# drop subtly useless features 
# writing skills in daily life
train_df.drop(['v253', 'v132', 'v284', 'v260', 'v26', 'v186', 'v162'], axis=1, inplace=True) 
# 380 - 11 = 369
print(train_df.shape)

(20000, 369)


In [47]:
# (2) drop sparse features 
# Borrowed from Stack Overflow QID: 26266362
def missing_values_table(df):
  mis_val = df.isnull().sum()
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  mis_val_table_ren_columns = mis_val_table.rename(
  columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
  '% of Total Values', ascending=False).round(1)
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
      "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
  return mis_val_table_ren_columns

return_t = missing_values_table(train_df)
return_t = return_t.loc[return_t['% of Total Values'] > 50.0]
return_t.head()
index_list = return_t.index.tolist() 
print(len(index_list))

train_df.drop(index_list, axis=1, inplace=True)
# 369 - 159 = 210
print(train_df.shape)

Your selected dataframe has 369 columns.
There are 355 columns that have missing values.
159
(20000, 210)


In [48]:
print(train_df)

                  cntryid  age_r gender_r  ...    v224     v71    v105
0                 Germany    NaN     Male  ...  9999.0  9996.0  9999.0
1          United Kingdom   35.0     Male  ...  9996.0  7110.0  9996.0
2           United States    NaN     Male  ...  9996.0  9999.0  9996.0
3                 Germany    NaN     Male  ...  9996.0  9999.0  9996.0
4                  France   32.0   Female  ...  5230.0  9996.0  4711.0
5           United States    NaN   Female  ...  9996.0  9999.0  9996.0
6                  Poland   36.0     Male  ...  9996.0  2512.0  9996.0
7                 Germany    NaN     Male  ...  9996.0  9999.0  9996.0
8          United Kingdom   40.0     Male  ...  9996.0  5819.0  9996.0
9                   Italy   38.0     Male  ...  9996.0  2220.0  9996.0
10                  Japan   40.0     Male  ...  9996.0  2610.0  9996.0
11          United States    NaN     Male  ...  9996.0  9999.0  9996.0
12                  Japan   43.0   Female  ...  9996.0  9996.0  9996.0
13    

In [49]:
# (3) Convert categorical features
train_df.dtypes

cntryid                object
age_r                 float64
gender_r               object
computerexperience     object
nativespeaker          object
edlevel3               object
monthlyincpr           object
yearlyincpr            object
lng_home               object
cnt_brth               object
reg_tl2                object
lng_bq                 object
lng_ci                 object
yrsqual               float64
yrsqual_t             float64
yrsget                float64
ctryqual               object
birthrgn               object
nativelang             object
ctryrgn                object
imyrcat                object
ageg5lfs               object
ageg10lfs              object
ageg10lfs_t            object
edcat8                 object
edcat7                 object
edcat6                 object
leavedu               float64
fe12                   object
aetpop                 object
                       ...   
v218                   object
v267                   object
v171      

In [77]:
num_df = train_df.select_dtypes(exclude=['object']).copy()
obj_df = train_df.select_dtypes(include=['object']).copy()
print(num_df.shape)
print(obj_df.shape)
obj_df.head()

(20000, 32)
(20000, 178)


,cntryid,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,cnt_brth,reg_tl2,lng_bq,lng_ci,ctryqual,birthrgn,nativelang,ctryrgn,imyrcat,ageg5lfs,ageg10lfs,ageg10lfs_t,edcat8,edcat7,edcat6,fe12,aetpop,faet12,faet12jr,faet12njr,nfe12,fnfaet12,edwork,neet,nopaidworkever,paidwork12,iscoskil4,isic1l,isic2l,isic1c,isic2c,nfe12jr,...,v2,v25,v18,v216,v178,v282,v13,v233,v278,v103,v155,v152,v258,v111,v218,v267,v171,v14,v7,v240,v149,v228,v28,v237,v280,v175,v288,v15,v208,v43,v27,v114,v191,v170,v65,v57,v177,v69,v85,v50
0,Germany,Male,Yes,No,Low,NaN,NaN,999,NaN,99999,deu,deu,NaN,NaN,Test language same as native language,North America and Western Europe,NaN,Aged 16-19,24 or less,24 or less,NaN,"Lower secondary (ISCED 2, ISCED 3C short)","Lower secondary or less (ISCED 1,2, 3C short o...",Did not participate in FE,AET population,Did not participate in formal AET,Did not participate in formal AET for JR reasons,Did not participate in FE for NJR reasons,Did not participate in NFE,Did not participate in formal or non-formal AET,Not in education or work and has not participa...,Not currently employed and did not participate...,Has had paid work,Has had paid work during the 12 months precedi...,Semi-skilled white-collar occupations,I,56,9996,9996,Did not participate in NFE for JR reasons,...,Rarely or never,Never,Never,Rarely or never,Never,Never,Rarely or never,Never,Never,Never,Never,No,Yes,Less than once a month,Every day,Never,Never,Less than once a week but at least once a month,Never,Every day,Less than once a month,Yes,Yes,Every day,Every day,Never,Never,At least once a week but not every day,Never,Every day,To some extent,To a high extent,To some extent,Not at all,Very little,To a high extent,Never,Agree,Strongly agree,Strongly agree
1,United Kingdom,Male,Yes,Yes,High,50 to less than 75,50 to less than 75,eng,United Kingdom of Great Britain and Northern I...,UKK,eng,eng,NaN,North America and Western Europe,Test language same as native language,North America and Western Europe,Non-immigrants,Aged 35-39,35-44,35-44,Tertiary - bachelor/master/research degree (IS...,Tertiary - bachelor/master/research degree (IS...,Tertiary - bachelor/master/research degree (IS...,Did not participate in FE,AET population,Did not participate in formal AET,Did not participate in formal AET for JR reasons,Did not participate in FE for NJR reasons,Participated in NFE,Participated in formal and/or non-formal AET,In work only,Employed or participated in education or train...,Has had paid work,Has had paid work during the 12 months precedi...,Skilled occupations,9996,9996,M,71,Participated in NFE for JR reasons,...,At least once a week,At least once a week but not every day,At least once a week,At least once a week,Every day,At least once a week,At least once a week,Every day,Never,Every day,At least once a week but not every day,Yes,Yes,Less than once a week but at least once a month,Every day,At least once a week but not every day,Less than once a week but at least once a month,Less than once a week but at least once a month,Less than once a week but at least once a month,Less than once a week but at least once a month,Less than once a month,NaN,Yes,Every day,Every day,At least once a week but not every day,Less than once a week but at least once a month,Less than once a week but at least once a month,Never,Never,To a high extent,To a high extent,To a high extent,To a high extent,To a high extent,To a high extent,Never,Disagree,Neither agree nor disagree,Agree
2,United States,Male,Yes,Yes,Medium,NaN,NaN,999,NaN,NaN,eng,eng,NaN,NaN,Test language same as native language,North America and Western Europe,Non-immigrants,Aged 50-54,45-54,45-54,"Post-secondary, non-tertiary (ISCED 4A-B-C)","Post-secondary, non-tertiary (ISCED 4A-B-C)","Post-secondary, non-tertiary (ISCED 4A-B-C)",Did not participate in FE,AET population,Did not participate in formal AET,Did not participate in formal AET for JR reasons,Did not participate in FE for NJR rea

In [57]:
# impute null values for objects!
col_df = obj_df[obj_df.isnull().any(axis=1)]
col_list = list(col_df.columns) 
print(col_list)
print(len(col_list))

['cntryid', 'gender_r', 'computerexperience', 'nativespeaker', 'edlevel3', 'monthlyincpr', 'yearlyincpr', 'lng_home', 'cnt_brth', 'reg_tl2', 'lng_bq', 'lng_ci', 'ctryqual', 'birthrgn', 'nativelang', 'ctryrgn', 'imyrcat', 'ageg5lfs', 'ageg10lfs', 'ageg10lfs_t', 'edcat8', 'edcat7', 'edcat6', 'fe12', 'aetpop', 'faet12', 'faet12jr', 'faet12njr', 'nfe12', 'fnfaet12', 'edwork', 'neet', 'nopaidworkever', 'paidwork12', 'iscoskil4', 'isic1l', 'isic2l', 'isic1c', 'isic2c', 'nfe12jr', 'nfe12njr', 'fnfe12jr', 'fnfaet12jr', 'fnfaet12njr', 'paidwork5', 'earnhrdcl', 'earnhrbonusdcl', 'earnmthalldcl', 'earnflag', 'learnatwork_wle_ca', 'readytolearn_wle_ca', 'icthome_wle_ca', 'influence_wle_ca', 'planning_wle_ca', 'readhome_wle_ca', 'readwork_wle_ca', 'taskdisc_wle_ca', 'writhome_wle_ca', 'writwork_wle_ca', 'v59', 'v200', 'v31', 'v151', 'v90', 'v157', 'v74', 'v153', 'v256', 'v102', 'v101', 'v60', 'v285', 'v17', 'v94', 'v227', 'v225', 'v201', 'v205', 'v46', 'v122', 'v92', 'v88', 'v84', 'v270', 'v140', '

In [62]:
cols = col_list
# fill with the mode, not the mean! 
obj_df[cols]=obj_df[cols].fillna(obj_df.mode().iloc[0])
# print(obj_df)
obj_df[obj_df.isnull().any(axis=1)] 
# Imputing categorical values complete!

,cntryid,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,cnt_brth,reg_tl2,lng_bq,lng_ci,ctryqual,birthrgn,nativelang,ctryrgn,imyrcat,ageg5lfs,ageg10lfs,ageg10lfs_t,edcat8,edcat7,edcat6,fe12,aetpop,faet12,faet12jr,faet12njr,nfe12,fnfaet12,edwork,neet,nopaidworkever,paidwork12,iscoskil4,isic1l,isic2l,isic1c,isic2c,nfe12jr,...,v2,v25,v18,v216,v178,v282,v13,v233,v278,v103,v155,v152,v258,v111,v218,v267,v171,v14,v7,v240,v149,v228,v28,v237,v280,v175,v288,v15,v208,v43,v27,v114,v191,v170,v65,v57,v177,v69,v85,v50


In [63]:
obj_df["cntryid"] = obj_df["cntryid"].astype('category')
obj_df.dtypes

cntryid               category
gender_r                object
computerexperience      object
nativespeaker           object
edlevel3                object
monthlyincpr            object
yearlyincpr             object
lng_home                object
cnt_brth                object
reg_tl2                 object
lng_bq                  object
lng_ci                  object
ctryqual                object
birthrgn                object
nativelang              object
ctryrgn                 object
imyrcat                 object
ageg5lfs                object
ageg10lfs               object
ageg10lfs_t             object
edcat8                  object
edcat7                  object
edcat6                  object
fe12                    object
aetpop                  object
faet12                  object
faet12jr                object
faet12njr               object
nfe12                   object
fnfaet12                object
                        ...   
v155                    object
v152    

In [64]:
obj_df["cntryid"] = obj_df["cntryid"].cat.codes
obj_df.head()


,cntryid,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,cnt_brth,reg_tl2,lng_bq,lng_ci,ctryqual,birthrgn,nativelang,ctryrgn,imyrcat,ageg5lfs,ageg10lfs,ageg10lfs_t,edcat8,edcat7,edcat6,fe12,aetpop,faet12,faet12jr,faet12njr,nfe12,fnfaet12,edwork,neet,nopaidworkever,paidwork12,iscoskil4,isic1l,isic2l,isic1c,isic2c,nfe12jr,...,v2,v25,v18,v216,v178,v282,v13,v233,v278,v103,v155,v152,v258,v111,v218,v267,v171,v14,v7,v240,v149,v228,v28,v237,v280,v175,v288,v15,v208,v43,v27,v114,v191,v170,v65,v57,v177,v69,v85,v50
0,10,Male,Yes,No,Low,25 to less than 50,25 to less than 50,999,Russian Federation,99999,deu,deu,North America and Western Europe,Central and Eastern Europe,Test language same as native language,North America and Western Europe,Non-immigrants,Aged 16-19,24 or less,24 or less,"Upper secondary (ISCED 3A-B, C long)","Lower secondary (ISCED 2, ISCED 3C short)","Lower secondary or less (ISCED 1,2, 3C short o...",Did not participate in FE,AET population,Did not participate in formal AET,Did not participate in formal AET for JR reasons,Did not participate in FE for NJR reasons,Did not participate in NFE,Did not participate in formal or non-formal AET,Not in education or work and has not participa...,Not currently employed and did not participate...,Has had paid work,Has had paid work during the 12 months precedi...,Semi-skilled white-collar occupations,I,56,9996,9996,Did not participate in NFE for JR reasons,...,Rarely or never,Never,Never,Rarely or never,Never,Never,Rarely or never,Never,Never,Never,Never,No,Yes,Less than once a month,Every day,Never,Never,Less than once a week but at least once a month,Never,Every day,Less than once a month,Yes,Yes,Every day,Every day,Never,Never,At least once a week but not every day,Never,Every day,To some extent,To a high extent,To some extent,Not at all,Very little,To a high extent,Never,Agree,Strongly agree,Strongly agree
1,29,Male,Yes,Yes,High,50 to less than 75,50 to less than 75,eng,United Kingdom of Great Britain and Northern I...,UKK,eng,eng,North America and Western Europe,North America and Western Europe,Test language same as native language,North America and Western Europe,Non-immigrants,Aged 35-39,35-44,35-44,Tertiary - bachelor/master/research degree (IS...,Tertiary - bachelor/master/research degree (IS...,Tertiary - bachelor/master/research degree (IS...,Did not participate in FE,AET population,Did not participate in formal AET,Did not participate in formal AET for JR reasons,Did not participate in FE for NJR reasons,Participated in NFE,Participated in formal and/or non-formal AET,In work only,Employed or participated in education or train...,Has had paid work,Has had paid work during the 12 months precedi...,Skilled occupations,9996,9996,M,71,Participated in NFE for JR reasons,...,At least once a week,At least once a week but not every day,At least once a week,At least once a week,Every day,At least once a week,At least once a week,Every day,Never,Every day,At least once a week but not every day,Yes,Yes,Less than once a week but at least once a month,Every day,At least once a week but not every day,Less than once a week but at least once a month,Less than once a week but at least once a month,Less than once a week but at least once a month,Less than once a week but at least once a month,Less than once a month,Yes,Yes,Every day,Every day,At least once a week but not every day,Less than once a week but at least once a month,Less than once a week but at least once a month,Never,Never,To a high extent,To a high extent,To a high extent,To a high extent,To a high extent,To a high extent,Never,Disagree,Neither agree nor disagree,Agree
2,30,Male,Yes,Yes,Medium,25 to less than 50,25 to less than 50,999,Russian Federation,99999,eng,eng,North America and Western Europe,Central and Eastern Europe,Test language same as native language,North America and Western Europe,Non-immigrants,Aged 50-54,45-54,45-54,"Post-secondary, non-tertiary (ISCED 4A-B-C)","Post-seco

In [75]:
# label encoding for each column that has objects
cols = col_list
obj_df[cols] = obj_df[cols].astype('category')

for i in cols:
  obj_df[i] = obj_df[i].cat.codes

print(obj_df.head())
obj_df.dtypes
final_obj_df = obj_df

   cntryid  gender_r  computerexperience  nativespeaker  ...  v177  v69  v85  v50
0       10         1                   1              0  ...     4    0    3    3
1       29         1                   1              1  ...     4    1    2    0
2       30         1                   1              1  ...     0    1    1    1
3       10         1                   1              1  ...     4    4    3    0
4        9         0                   1              1  ...     4    3    3    3

[5 rows x 178 columns]


In [0]:
# impute NaNs for the datset with integer datatype to begin with 
num_df[num_df.isnull().any(axis=1)]
# fill with the mean! 
imputed_num_df = num_df.fillna(num_df.mean())

In [84]:
imputed_num_df[imputed_num_df.isnull().any(axis=1)]
# no NaNs -- complete


,age_r,yrsqual,yrsqual_t,yrsget,leavedu,isco1c,isco2c,isco1l,isco2l,learnatwork,readytolearn,icthome,influence,planning,readhome,readwork,taskdisc,writhome,writwork,v202,v231,v272,v52,v97,v242,v135,v235,v1,v239,v224,v71,v105


In [86]:
# join the two datasets
final_table = pd.concat([final_obj_df, imputed_num_df], axis=1)
print(final_table)
print(final_table.shape)


       cntryid  gender_r  computerexperience  ...    v224     v71    v105
0           10         1                   1  ...  9999.0  9996.0  9999.0
1           29         1                   1  ...  9996.0  7110.0  9996.0
2           30         1                   1  ...  9996.0  9999.0  9996.0
3           10         1                   1  ...  9996.0  9999.0  9996.0
4            9         0                   1  ...  5230.0  9996.0  4711.0
5           30         0                   1  ...  9996.0  9999.0  9996.0
6           21         1                   1  ...  9996.0  2512.0  9996.0
7           10         1                   0  ...  9996.0  9999.0  9996.0
8           29         1                   1  ...  9996.0  5819.0  9996.0
9           14         1                   1  ...  9996.0  2220.0  9996.0
10          15         1                   1  ...  9996.0  2610.0  9996.0
11          30         1                   1  ...  9996.0  9999.0  9996.0
12          15         0              

In [98]:
final_table[final_table.isnull().any(axis=1)]
# impute really complete!


,cntryid,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,cnt_brth,reg_tl2,lng_bq,lng_ci,ctryqual,birthrgn,nativelang,ctryrgn,imyrcat,ageg5lfs,ageg10lfs,ageg10lfs_t,edcat8,edcat7,edcat6,fe12,aetpop,faet12,faet12jr,faet12njr,nfe12,fnfaet12,edwork,neet,nopaidworkever,paidwork12,iscoskil4,isic1l,isic2l,isic1c,isic2c,nfe12jr,...,v191,v170,v65,v57,v177,v69,v85,v50,age_r,yrsqual,yrsqual_t,yrsget,leavedu,isco1c,isco2c,isco1l,isco2l,learnatwork,readytolearn,icthome,influence,planning,readhome,readwork,taskdisc,writhome,writwork,v202,v231,v272,v52,v97,v242,v135,v235,v1,v239,v224,v71,v105


In [99]:
X = final_table
# (5) Train on 75% of data, validation 25% 
num = int(len(X)*(75/100))
# print(num)
X_train = X.head(num)
X_val = X.head(len(X)-num)
print(len(X_train))
print(len(X_val))

# get Y
Y_train = Y.head(num)
Y_val = Y.tail(len(Y)-num)
print(len(Y_train))
print(len(Y_val))

# reshape
X_train = X_train.values.reshape(-1, 1)
Y_train = Y_train.values.reshape(-1, 1)

X_val = X_val.values.reshape(-1, 1)
Y_val = Y_val.values.reshape(-1, 1)


15000
5000
15000
5000


In [100]:
# trying kernel ridge regression bc lots of parameters + reducing colinearity 
# starter code borrowed from skikit-learn documentation on SKR
from sklearn.kernel_ridge import KernelRidge
# n_samples, n_features = 20000, 210
regressor = KernelRidge(alpha=1.0)
regressor.fit(X_train, Y_train)


ValueError: ignored

In [69]:
# Dropping NaNs (change later on)
print(train_df.shape)
print(train_df.shape)
X = train_df.loc[:, 'age_r']
Y = test_df[:, 'job_performance']
print(type(X))
print(type(Y))
# X = X.values.reshape(-1, 1)
# Y = Y.values.reshape(-1, 1)
# print(type(X))
# print(type(Y))

# X = X.dropna()
# Y = Y.dropna()
# X = X.values.reshape(-1, 1)
# Y = Y.values.reshape(-1, 1)

nanArray = np.argwhere(np.isnan(X))
nanArray = nanArray[:, 0]
print(nanArray)
print(len(nanArray))

Y = Y.values.reshape(-1, 1)
print(type(Y))
for count, val in enumerate(Y):
#   print (count, val)
  if count in nanArray:
    Y[count] = 0 # instead of deleting the val, which may shift the index
    
print(len(Y))
Y = Y[Y!= 0]
print(20000-len(Y))

X = X.dropna()
X = X.values.reshape(-1, 1)
print(type(X))
print(type(Y))
print(len(X))
print(len(Y))

lm = linear_model.LinearRegression()
reg = lm.fit(X,Y)
print(reg.score(X, Y))
predictions = lm.predict(X)
print(predictions[0:10])
print(Y[0:10])



(20000, 380)
(20000, 380)
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
[    0     5     7 ... 19988 19989 19995]
5487
<class 'numpy.ndarray'>
20000
5487
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
14513
14513
0.019078110398784265
[2500.64710853 2576.48829498 2723.11458879 2596.71261137 2707.9463515
 2611.88084866 2667.49771872 2657.38556053 2515.81534582 2556.2639786 ]
[1478.76044746 2347.43097473 3302.63889441 2380.64957602 2622.46923816
 3293.46978793 2787.83424227 2910.22545184 2241.25670706 2992.87834534]


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


In [0]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_true, y_pred)
print(mse)

In [0]:
# preprocess test_set the same way
list of data 

In [0]:
# run on test_set, put it back to CSV file 